### <font color='red'>Cardiac Arrhythmia Multi-Class Classification 

# Process Flow :

Analyze data and address missing data if there is any. 

Decide aboute a good evaluation strategy and justify your choice. 

Find the best parameters for the following classification models: 
- KNN classifcation 
- Logistic Regression
- Linear Supprt Vector Machine
- Kerenilzed Support Vector Machine
- Decision Tree
- Random Forest 

Then use different bagging and boosting methods to boost the results. Do you see any significant change? 

use data reduction method to reduce the size of data, and agian try above models. Do you get better results? 

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import GridSearchCV
import warnings
warnings.filterwarnings('ignore')
from sklearn.metrics import recall_score
from sklearn.model_selection import StratifiedKFold
kFold = StratifiedKFold(n_splits=5)

## <font color='red'> Reading data

In [2]:
data = pd.read_csv('cardiac_arrhythmia.csv',header=None)

In [3]:
data.head()

,0,1,2,3,4,5,6,7,8,9,...,270,271,272,273,274,275,276,277,278,279
0,75,0,190,80,91,193,371,174,121,-16,...,0.0,9.0,-0.9,0.0,0,0.9,2.9,23.3,49.4,8
1,56,1,165,64,81,174,401,149,39,25,...,0.0,8.5,0.0,0.0,0,0.2,2.1,20.4,38.8,6
2,54,0,172,95,138,163,386,185,102,96,...,0.0,9.5,-2.4,0.0,0,0.3,3.4,12.3,49.0,10
3,55,0,175,94,100,202,380,179,143,28,...,0.0,12.2,-2.2,0.0,0,0.4,2.6,34.6,61.6,1
4,75,0,190,80,88,181,360,177,103,-16,...,0.0,13.1,-3.6,0.0,0,-0.1,3.9,25.4,62.8,7


In [4]:
data.shape

(452, 280)

In [5]:
data[279]

0       8
1       6
2      10
3       1
4       7
5      14
6       1
7       1
8       1
9      10
10      3
11      1
12     10
13      6
14      1
15      1
16     10
17      1
18      1
19      1
20      1
21      1
22      1
23      1
24      1
25     16
26     14
27     10
28      2
29      2
       ..
422     1
423     1
424     9
425     1
426     1
427    10
428     1
429    16
430    10
431     6
432    10
433     3
434     1
435     1
436     1
437     1
438     1
439     1
440     1
441     1
442     1
443    10
444     1
445     1
446     1
447     1
448    10
449     2
450     1
451     1
Name: 279, Length: 452, dtype: int64

## <font color='red'> Handling Missing Data

### Counting missing data

In [6]:
count=0
for i in range(0,452):
    for j in range(0,280):
        if (data.iloc[i,j]=='?'):
            count =count+1
print(count)

408


### Replacing '?' with Null 

In [7]:
for i in range(0,452):
    for j in range(0,280):
        if (data.iloc[i,j]=='?'):
            data.iloc[i,j] = np.nan

In [8]:
pd.isnull(data).sum()

0        0
1        0
2        0
3        0
4        0
5        0
6        0
7        0
8        0
9        0
10       8
11      22
12       1
13     376
14       1
15       0
16       0
17       0
18       0
19       0
20       0
21       0
22       0
23       0
24       0
25       0
26       0
27       0
28       0
29       0
      ... 
250      0
251      0
252      0
253      0
254      0
255      0
256      0
257      0
258      0
259      0
260      0
261      0
262      0
263      0
264      0
265      0
266      0
267      0
268      0
269      0
270      0
271      0
272      0
273      0
274      0
275      0
276      0
277      0
278      0
279      0
Length: 280, dtype: int64

### Droping column 13 with lots of missing values

In [9]:
data.drop(columns = 13, inplace=True)

### Imputing other missing data using KNN

In [10]:
import fancyimpute
data_no_missing = fancyimpute.KNN(k=5).complete(data)

Using TensorFlow backend.


Imputing row 1/452 with 0 missing, elapsed time: 0.432
Imputing row 101/452 with 0 missing, elapsed time: 0.432
Imputing row 201/452 with 1 missing, elapsed time: 0.432
Imputing row 301/452 with 1 missing, elapsed time: 0.432
Imputing row 401/452 with 0 missing, elapsed time: 0.436


In [11]:
pd.isnull(data_no_missing).sum()

0

In [12]:
data_no_missing = pd.DataFrame(data_no_missing)

In [13]:
data_no_missing.head()

,0,1,2,3,4,5,6,7,8,9,...,269,270,271,272,273,274,275,276,277,278
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4,8.0
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8,6.0
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0,10.0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6,1.0
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8,7.0


## <font color='red'> Data Exploration

In [14]:
data_no_missing[278].value_counts()

1.0     245
10.0     50
2.0      44
6.0      25
16.0     22
4.0      15
3.0      15
5.0      13
9.0       9
15.0      5
14.0      4
7.0       3
8.0       2
Name: 278, dtype: int64

In [15]:
data_no_missing.corr()

,0,1,2,3,4,5,6,7,8,9,...,269,270,271,272,273,274,275,276,277,278
0,1.000000,-0.059042,-0.109458,0.381555,-0.004032,0.041149,0.195691,0.025654,0.099755,-0.265868,...,0.164816,-0.158009,0.082376,0.090413,NaN,-0.037877,-0.271504,0.018043,-0.199728,-0.092381
1,-0.059042,1.000000,-0.124685,-0.248104,-0.337101,-0.046771,0.072052,-0.184736,-0.081051,0.069434,...,0.230938,-0.042638,0.092879,0.027401,NaN,0.014210,0.065684,0.031688,0.046605,-0.178080
2,-0.109458,-0.124685,1.000000,-0.074957,-0.006329,0.013601,-0.237314,-0.038411,0.029025,0.061539,...,-0.018876,-0.073439,-0.091361,-0.002545,NaN,0.067670,-0.008471,-0.090370,-0.092235,0.006648
3,0.381555,-0.248104,-0.074957,1.000000,0.100094,0.119826,0.118657,0.149987,0.120668,-0.173355,...,0.050471,-0.026926,0.051946,0.047448,NaN,-0.046278,-0.144721,0.062285,-0.050682,-0.090151
4,-0.004032,-0.337101,-0.006329,0.100094,1.000000,0.021831,0.218681,0.397435,0.049682,-0.146043,...,-0.198941,0.091384,-0.228688,-0.013697,NaN,-0.065596,-0.222170,0.129796,-0.082791,0.323879
5,0.041149,-0.046771,0.013601,0.119826,0.021831,1.000000,0.079371,0.074618,0.670865,-0.012412,...,-0.005347,0.003411,-0.074180,0.068594,NaN,0.143272,0.061160,-0.027691,0.021048,-0.099954
6,0.195691,0.072052,-0.237314,0.118657,0.218681,0.079371,1.000000,0.166711,0.063044,-0.031786,...,0.035215,0.121349,0.116718,-0.002351,NaN,-0.034971,-0.038884,0.256200,0.150979,0.028305
7,0.025654,-0.184736,-0.038411,0.149987,0.397435,0.074618,0.166711,1.000000,0.060302,-0.099283,...,-0.068035,0.070535,-0.046868,0.008556,NaN,0.049175,-0.184846,0.130202,-0.014430,0.097625
8,0.099755,-0.081051,0.029025,0.120668,0.049682,0.670865,0.063044,0.060302,1.000000,-0.062760,...,-0.010142,0.007886,-0.091258,0.096772,NaN,0.248092,0.016054,-0.016365,0.004283,-0.122003
9,-0.265868,0.069434,0.061539,-0.173355,-0.146043,-0.012412,-0.031786,-0.099283,-0.062760,1.000000,...,-0.179725,0.292155,0.295283,-0.071495,NaN,0.082009,0.063300,0.297780,0.255203,0.019585


- We can see there is no major correlation between variables

## <font color='red'> Data Preparation

### X, y split

In [16]:
X = data_no_missing.drop(columns=278)
X.head()

,0,1,2,3,4,5,6,7,8,9,...,268,269,270,271,272,273,274,275,276,277
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8


In [17]:
X_columns=["Age","Gender_Nom","Height","Weight","QRS_Dur",
"P-R_Int","Q-T_Int","T_Int","P_Int","QRS","T","P","J","Heart_Rate",
"Q_Wave","R_Wave","S_Wave","R_Prime","S_Prime","Int_Def","Rag_R_Nom",
"Diph_R_Nom","Rag_P_Nom","Diph_P_Nom","Rag_T_Nom","Diph_T_Nom", 
"DII00", "DII01","DII02", "DII03", "DII04","DII05","DII06","DII07","DII08","DII09","DII10","DII11",
"DIII00","DIII01","DIII02", "DIII03", "DIII04","DIII05","DIII06","DIII07","DIII08","DIII09","DIII10","DIII11",
"AVR00","AVR01","AVR02","AVR03","AVR04","AVR05","AVR06","AVR07","AVR08","AVR09","AVR10","AVR11",
"AVL00","AVRL1","AVL02","AVL03","AVL04","AVL05","AVL06","AVL07","AVL08","AVL09","AVL10","AVL11",
"AVF00","AVF01","AVF02","AVF03","AVF04","AVF05","AVF06","AVF07","AVF08","AVF09","AVF10","AVF11",
"V100","V101","V102","V103","V104","V105","V106","V107","V108","V109","V110","V111",
"V200","V201","V202","V203","V204","V205","V206","V207","V208","V209","V210","V211",
"V300","V301","V302","V303","V304","V305","V306","V307","V308","V309","V310","V311",
"V400","V401","V402","V403","V404","V405","V406","V407","V408","V409","V410","V411",
"V500","V501","V502","V503","V504","V505","V506","V507","V508","V509","V510","V511",
"V600","V601","V602","V603","V604","V605","V606","V607","V608","V609","V610","V611",
"JJ_Wave","Q_Wave","R_Wave","S_Wave","R_Prime_Wave","S_Prime_Wave","P_Wave","T_Wave",
"QRSA","QRSTA","DII170","DII171","DII172","DII173","DII174","DII175","DII176","DII177","DII178","DII179",
"DIII180","DIII181","DIII182","DIII183","DIII184","DIII185","DIII186","DIII187","DIII188","DIII189",
"AVR190","AVR191","AVR192","AVR193","AVR194","AVR195","AVR196","AVR197","AVR198","AVR199",
"AVL200","AVL201","AVL202","AVL203","AVL204","AVL205","AVL206","AVL207","AVL208","AVL209",
"AVF210","AVF211","AVF212","AVF213","AVF214","AVF215","AVF216","AVF217","AVF218","AVF219",
"V1220","V1221","V1222","V1223","V1224","V1225","V1226","V1227","V1228","V1229",
"V2230","V2231","V2232","V2233","V2234","V2235","V2236","V2237","V2238","V2239",
"V3240","V3241","V3242","V3243","V3244","V3245","V3246","V3247","V3248","V3249",
"V4250","V4251","V4252","V4253","V4254","V4255","V4256","V4257","V4258","V4259",
"V5260","V5261","V5262","V5263","V5264","V5265","V5266","V5267","V5268","V5269",
"V6270","V6271","V6272","V6273","V6274","V6275","V6276","V6277","V6278","V6279"]

In [18]:
X.columns = X_columns
X.head()

,Age,Gender_Nom,Height,Weight,QRS_Dur,P-R_Int,Q-T_Int,T_Int,P_Int,QRS,...,V6270,V6271,V6272,V6273,V6274,V6275,V6276,V6277,V6278,V6279
0,75.0,0.0,190.0,80.0,91.0,193.0,371.0,174.0,121.0,-16.0,...,-0.3,0.0,9.0,-0.9,0.0,0.0,0.9,2.9,23.3,49.4
1,56.0,1.0,165.0,64.0,81.0,174.0,401.0,149.0,39.0,25.0,...,-0.5,0.0,8.5,0.0,0.0,0.0,0.2,2.1,20.4,38.8
2,54.0,0.0,172.0,95.0,138.0,163.0,386.0,185.0,102.0,96.0,...,0.9,0.0,9.5,-2.4,0.0,0.0,0.3,3.4,12.3,49.0
3,55.0,0.0,175.0,94.0,100.0,202.0,380.0,179.0,143.0,28.0,...,0.1,0.0,12.2,-2.2,0.0,0.0,0.4,2.6,34.6,61.6
4,75.0,0.0,190.0,80.0,88.0,181.0,360.0,177.0,103.0,-16.0,...,-0.4,0.0,13.1,-3.6,0.0,0.0,-0.1,3.9,25.4,62.8


In [19]:
y = data_no_missing[278]
y.head()

0     8.0
1     6.0
2    10.0
3     1.0
4     7.0
Name: 278, dtype: float64

In [20]:
y.columns = ["Class"]
y.head()

0     8.0
1     6.0
2    10.0
3     1.0
4     7.0
Name: 278, dtype: float64

### Train-test split

In [21]:
from sklearn.model_selection import train_test_split
X_train_org, X_test_org, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state = 10)

In [22]:
X_test_org.head()

,Age,Gender_Nom,Height,Weight,QRS_Dur,P-R_Int,Q-T_Int,T_Int,P_Int,QRS,...,V6270,V6271,V6272,V6273,V6274,V6275,V6276,V6277,V6278,V6279
303,67.0,0.0,176.0,80.0,97.0,144.0,357.0,170.0,100.0,-48.0,...,-0.2,0.0,3.6,-2.6,0.0,0.0,1.0,1.5,0.2,14.6
131,48.0,1.0,165.0,70.0,83.0,146.0,418.0,134.0,83.0,48.0,...,-1.0,0.0,11.9,0.0,0.0,0.0,1.0,0.6,28.5,31.7
116,74.0,0.0,172.0,74.0,106.0,165.0,411.0,161.0,96.0,55.0,...,-0.5,-0.5,6.5,0.0,0.0,0.0,0.3,2.6,24.2,48.6
305,14.0,0.0,175.0,59.0,96.0,141.0,340.0,225.0,87.0,80.0,...,0.1,-0.8,14.3,-2.5,0.0,0.0,0.5,3.4,24.8,57.4
65,44.0,1.0,155.0,65.0,80.0,117.0,363.0,142.0,72.0,56.0,...,-0.2,0.0,6.2,0.0,0.0,0.0,0.5,1.1,16.1,23.8


### Scaling using MinMaxScaler

In [23]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train_org)
X_test = scaler.transform(X_test_org)

## <font color='red'> Machine Learning Models (Without Bagging Boosting and PCA)

- We will be using "WEIGHTED RECALL" as evaluation strategy because we are predicting Cardiac Arrhythmia, which is serious medical condition.
- We don't want to misclassify someone having arrhythmia as normal. It is lot bigger risk than classifying someone normal as having arrhythmia. So we want to maximize true positive rate i.e. Recall.
- Weighted recall is used instead of normal recall because it accounts for label imbalance present in data.

### KNN clasiification

In [24]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_jobs=-1)

param_grid={'n_neighbors':[1,2,3,4,5,7,10]}

grid_search = GridSearchCV(knn_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 7, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [25]:
grid_search.cv_results_

{'mean_fit_time': array([0.00762138, 0.00780034, 0.00873671, 0.00838623, 0.0072885 ,
        0.00823112, 0.00707488]),
 'mean_score_time': array([0.11858268, 0.11493788, 0.1162931 , 0.11709194, 0.113901  ,
        0.11549411, 0.11699977]),
 'mean_test_score': array([0.55208333, 0.59114583, 0.59635417, 0.578125  , 0.57291667,
        0.5625    , 0.56770833]),
 'mean_train_score': array([1.        , 0.68618421, 0.66540019, 0.63084585, 0.62170995,
        0.60026961, 0.57163885]),
 'param_n_neighbors': masked_array(data=[1, 2, 3, 4, 5, 7, 10],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1},
  {'n_neighbors': 2},
  {'n_neighbors': 3},
  {'n_neighbors': 4},
  {'n_neighbors': 5},
  {'n_neighbors': 7},
  {'n_neighbors': 10}],
 'rank_test_score': array([7, 2, 1, 3, 4, 6, 5]),
 'split0_test_score': array([0.57317073, 0.58536585, 0.59756098, 0.54878049, 0.53658537,
        0.53658537, 0.5243

In [26]:
grid_search.best_params_

{'n_neighbors': 3}

- Best Parameter : No. of neighbours=3

In [27]:
grid_search.best_score_

0.5963541666666666

In [28]:
knn_clf=KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [29]:
y_pred_train = knn_clf.predict(X_train)
y_pred_test = knn_clf.predict(X_test)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.6692708333333334
Test Recall score: 0.6470588235294118


- We can see that test score is poor and KNN doesn't perform well.
- But model is a good fit as train and test scores are almost same.

### Logistic Regression

In [30]:
from sklearn.linear_model import LogisticRegression

lreg_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

param_grid = {'C': [0.00001,0.0001,0.001,0.01,0.1,1,10,100]}

grid_search = GridSearchCV(lreg_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [31]:
grid_search.cv_results_

{'mean_fit_time': array([0.20082269, 0.14297881, 0.18642406, 0.18857894, 0.18225813,
        0.20940337, 0.2022018 , 0.19891438]),
 'mean_score_time': array([0.00050845, 0.00121174, 0.00120168, 0.00104451, 0.00242391,
        0.00106215, 0.00111279, 0.00150518]),
 'mean_test_score': array([0.53385417, 0.53385417, 0.53385417, 0.53385417, 0.609375  ,
        0.70052083, 0.68489583, 0.66666667]),
 'mean_train_score': array([0.53393884, 0.53393884, 0.53393884, 0.53393884, 0.64196967,
        0.83792892, 0.96622393, 0.99481065]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001},
  {'C': 0.01},
  {'C': 0.1},
  {'C': 1},
  {'C': 10},
  {'C': 100}],
 'rank_test_score': array([5, 5, 5, 5, 4, 1, 2, 3]),
 'split0_test_score': array([0.5       , 0.5       , 0.5       , 0.5       , 0.597

In [32]:
grid_search.best_params_

{'C': 1}

- Best parameter : C=1

In [33]:
grid_search.best_score_

0.7005208333333334

In [34]:
lreg_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=1)
lreg_clf.fit(X_train, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [35]:
y_pred_train = lreg_clf.predict(X_train)
y_pred_test = lreg_clf.predict(X_test)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.8411458333333334
Test Recall score: 0.6617647058823529


- We can see that test score is poor and logistic regression doesn't perform well.
- Also model is overfitting as there is large difference between train and test score.

### Linear SVM

In [36]:
from sklearn.svm import LinearSVC

LSVC_clf = LinearSVC(multi_class='crammer_singer')

param_grid = {'C': [0.00001,0.0001,0.001,0.01,0.1,1,10,100]}

grid_search = GridSearchCV(LSVC_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [37]:
grid_search.cv_results_

{'mean_fit_time': array([0.02250376, 0.06714101, 0.13118382, 0.26424427, 0.66457653,
        2.30589085, 6.66223407, 6.06002626]),
 'mean_score_time': array([0.00296249, 0.0014688 , 0.00118303, 0.0014904 , 0.00101552,
        0.00073786, 0.00130463, 0.00296283]),
 'mean_test_score': array([0.53385417, 0.53385417, 0.53385417, 0.578125  , 0.69791667,
        0.69791667, 0.609375  , 0.60677083]),
 'mean_train_score': array([0.53393884, 0.53393884, 0.53393884, 0.60413872, 0.78261155,
        0.94141337, 0.99871795, 1.        ]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001},
  {'C': 0.01},
  {'C': 0.1},
  {'C': 1},
  {'C': 10},
  {'C': 100}],
 'rank_test_score': array([6, 6, 6, 5, 1, 1, 3, 4]),
 'split0_test_score': array([0.5       , 0.5       , 0.5       , 0.53658537, 0.682

In [38]:
grid_search.best_params_

{'C': 0.1}

- Best parameter : C=0.1

In [39]:
grid_search.best_score_

0.6979166666666666

In [40]:
LSVC_clf = LinearSVC(multi_class='crammer_singer', C=0.1)
LSVC_clf.fit(X_train, y_train)

LinearSVC(C=0.1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)

In [41]:
y_pred_train = LSVC_clf.predict(X_train)
y_pred_test = LSVC_clf.predict(X_test)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.7838541666666666
Test Recall score: 0.7205882352941176


- We can see that test score is better than KNN and logistic regression, so Linear SVC performs better.
- Also model is good fit as there is not much difference between train and test score.

### Kernelised SVM

In [42]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn import svm

KSVC_clf = svm.SVC(kernel='rbf')

param_grid = {'C': [0.0001,0.001,0.01,0.1,1,10],
          'gamma': [0.0001,0.001,0.01,0.1,1,10]}

grid_search = GridSearchCV(KSVC_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [43]:
grid_search.cv_results_

{'mean_fit_time': array([0.06030979, 0.06716127, 0.07048559, 0.05828342, 0.06091447,
        0.06661263, 0.06041207, 0.07309465, 0.07571621, 0.07810373,
        0.07388053, 0.0863554 , 0.06987319, 0.0632894 , 0.05978227,
        0.09247537, 0.17898526, 0.24360237, 0.0641448 , 0.06684971,
        0.08680263, 0.10804205, 0.274931  , 0.33121161, 0.07830763,
        0.07529593, 0.09669714, 0.09915004, 0.21826315, 0.28718529,
        0.08964982, 0.10403247, 0.10201068, 0.12808509, 0.30487986,
        0.35476756]),
 'mean_score_time': array([0.01279259, 0.01532097, 0.01614127, 0.01207814, 0.01284089,
        0.01280828, 0.01331205, 0.01491227, 0.01588969, 0.01480074,
        0.0166368 , 0.01542611, 0.01429491, 0.01331658, 0.01229892,
        0.01568351, 0.01738601, 0.01872816, 0.01286259, 0.01286073,
        0.01290569, 0.01487541, 0.01890073, 0.01810036, 0.01577387,
        0.01214061, 0.01354046, 0.01473722, 0.01436176, 0.01635094,
        0.01562152, 0.01779146, 0.0149385 , 0.01702042, 0.

In [44]:
grid_search.best_params_

{'C': 10, 'gamma': 0.1}

- Best parameters : C=10, gamma=0.1

In [45]:
grid_search.best_score_

0.6979166666666666

In [46]:
from sklearn import svm
KSVC_clf = svm.SVC(kernel='rbf',C=10,gamma=0.1)

KSVC_clf.fit(X_train, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [47]:
y_pred_train = KSVC_clf.predict(X_train)
y_pred_test = KSVC_clf.predict(X_test)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.9765625
Test Recall score: 0.6764705882352942


- We can see that test score is poor and Kernalised SVM doesn't perform well.
- Also model is overfitting as there is large difference between train and test score.

### Decision Tree

In [48]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()
param_grid = {'max_depth': [2, 3, 4, 5,6, 10, 20]}

grid_search = GridSearchCV(dt_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4, 5, 6, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [49]:
grid_search.cv_results_

{'mean_fit_time': array([0.01182098, 0.01528044, 0.02179918, 0.02717204, 0.02417645,
        0.04401808, 0.06752858]),
 'mean_score_time': array([0.00226817, 0.00100346, 0.00166602, 0.00141649, 0.00120249,
        0.00150499, 0.00250907]),
 'mean_test_score': array([0.6171875 , 0.65104167, 0.67447917, 0.65625   , 0.67447917,
        0.65625   , 0.66666667]),
 'mean_train_score': array([0.64791021, 0.71820893, 0.77155467, 0.81909989, 0.8627067 ,
        0.96624575, 1.        ]),
 'param_max_depth': masked_array(data=[2, 3, 4, 5, 6, 10, 20],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2},
  {'max_depth': 3},
  {'max_depth': 4},
  {'max_depth': 5},
  {'max_depth': 6},
  {'max_depth': 10},
  {'max_depth': 20}],
 'rank_test_score': array([7, 6, 1, 4, 1, 4, 3]),
 'split0_test_score': array([0.56097561, 0.6097561 , 0.63414634, 0.62195122, 0.67073171,
        0.65853659, 0.70731707]),
 'spli

In [50]:
grid_search.best_params_

{'max_depth': 4}

- Best parameter : max_depth = 4

In [51]:
grid_search.best_score_

0.6744791666666666

In [52]:
dt_clf = DecisionTreeClassifier(max_depth=4)
dt_clf.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [53]:
y_pred_train = dt_clf.predict(X_train)
y_pred_test = dt_clf.predict(X_test)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.75
Test Recall score: 0.6764705882352942


- We can see that test score is poor and decision tree doesn't perform well.
- Also model is somewhat overfitting as there is difference between train and test score.

### Random Forest

In [54]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=300, criterion='gini',n_jobs= -1,random_state=10)

param_grid = {'max_features': [100,125,150,200],
          'max_depth': [6,8,10,12,14],
           'max_leaf_nodes':[20,22,30,50]}

grid_search = GridSearchCV(rf_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=-1,
            oob_score=False, random_state=10, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_features': [100, 125, 150, 200], 'max_depth': [6, 8, 10, 12, 14], 'max_leaf_nodes': [20, 22, 30, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [55]:
grid_search.cv_results_

{'mean_fit_time': array([1.05817866, 1.13703184, 0.96248488, 1.03218384, 1.34079223,
        1.37787457, 1.24069099, 1.21030602, 1.33798671, 1.3563683 ,
        1.45740261, 1.56993146, 1.57798958, 1.62058187, 1.8045126 ,
        1.63484211, 1.30532136, 1.31167459, 1.31384811, 1.24995799,
        1.49451947, 1.45090532, 1.52943316, 1.406915  , 1.61174874,
        1.64421811, 1.66761022, 1.59551539, 1.92139764, 1.99900608,
        1.97438889, 1.98496685, 1.39027963, 1.32272472, 1.16238484,
        1.16500826, 1.38323421, 1.32395477, 1.28452144, 1.25006595,
        1.34128418, 1.42402616, 1.47317944, 1.4259654 , 1.73890505,
        1.67893476, 1.84885297, 1.66389675, 1.10986547, 1.19443517,
        1.16203184, 1.25804796, 1.33466763, 1.28418012, 1.32359052,
        1.44121981, 1.28083091, 1.30033045, 1.52439909, 1.32607722,
        1.47940273, 1.61456137, 1.91717005, 1.64939094, 1.06808796,
        1.11333671, 1.19456391, 1.32355509, 1.16075969, 1.30092664,
        1.33680553, 1.46544747,

In [56]:
grid_search.best_params_

{'max_depth': 10, 'max_features': 100, 'max_leaf_nodes': 30}

- Best parameters : max_features=100, max_depth=10, max_leaf_nodes=30

In [57]:
grid_search.best_score_

0.7578125

In [58]:
rf_clf = RandomForestClassifier(n_estimators=300, criterion='gini',max_features=100,max_depth=10,max_leaf_nodes=30)
rf_clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features=100, max_leaf_nodes=30,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=300, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [59]:
y_pred_train = rf_clf.predict(X_train)
y_pred_test = rf_clf.predict(X_test)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.9557291666666666
Test Recall score: 0.7647058823529411


- We can see that test score is good and Random forest performs well as it is ensemble method.
- But model is overfitting as there is large difference between train and test score.

## <font color='red'> Voting Classifier

In [60]:
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

hard_voting_clf = VotingClassifier(estimators=[('knn', knn_clf),('lr',lreg_clf),('lsvc', LSVC_clf),
                                   ('ksvc', KSVC_clf),('dt', dt_clf), ('rt', rf_clf)],voting = 'hard')
hard_voting_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(hard_voting_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(hard_voting_clf.score(X_test, y_test)))

Train score: 0.85
Test score: 0.69


In [61]:
score = cross_val_score(estimator=hard_voting_clf,X=X_train,y=y_train, scoring='recall_weighted', cv=kFold)

In [62]:
print('Mean Score: {0:0.2f}'.format(score.mean()))
print('Mean Std: {0:0.2f}'.format(score.std()))

Mean Score: 0.71
Mean Std: 0.02


In [63]:
X_scaled = scaler.fit_transform(X)

## <font color='red'> Machine Learning Models with Bagging and/or Boosting

### Bagging with KNN

In [64]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
bag_knn = BaggingClassifier(base_estimator=knn_clf, n_estimators=100,bootstrap_features=True, max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_knn, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)

In [65]:
print('Mean score:', score.mean())

Mean score: 0.5448364251823543


Here, Bagging is giving us a very low score. It doesn't improve our model

### Pasting with KNN

In [66]:
from sklearn.ensemble import BaggingClassifier
from sklearn.neighbors import KNeighborsClassifier

#knn = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
bag_knn = BaggingClassifier(base_estimator=knn_clf, n_estimators=100,bootstrap_features=True, bootstrap=False,
                            max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_knn, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)

In [67]:
print('Mean score:', score.mean())

Mean score: 0.5448364251823543


Pasting is also giving us a very low score. It doesn't improve our model

### Bagging with Logistic Reg

In [68]:
from sklearn.ensemble import BaggingClassifier

bag_log = BaggingClassifier(base_estimator=lreg_clf, n_estimators=100,bootstrap_features=True, max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_log, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)
print('Mean score:', score.mean())

Mean score: 0.5493598087755054


The bagging algorithm decreased the variance, but raised the bias

### Bagging with Linear SVC

In [69]:
bag_lsvc = BaggingClassifier(base_estimator=LSVC_clf, n_estimators=100,bootstrap_features=True, max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_lsvc, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)
print('Mean score:', score.mean())

Mean score: 0.6398795000639281


The bagging algorithm has raised the bias and doesn't improve our model

### Bagging with SVM 

In [70]:
bag_ksvc = BaggingClassifier(base_estimator=KSVC_clf, n_estimators=100,bootstrap_features=True, max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_ksvc, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)
print('Mean score:', score.mean())

Mean score: 0.5556073763471118


The bagging algorithm has raised the bias and doesn't improve our model

### Bagging with Decision Tree

In [73]:
bag_dt = BaggingClassifier(base_estimator=dt_clf, n_estimators=100,bootstrap_features=True, max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_dt, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)
print('Mean score:', score.mean())

Mean score: 0.6156329272465049


The bagging algorithm has raised the bias and doesn't improve our model

### Bagging with Random Forest

In [74]:
bag_rf = BaggingClassifier(base_estimator=rf_clf, n_estimators=100,bootstrap_features=True, max_samples=50, max_features=100)

score = cross_val_score(estimator=bag_rf, X=X_scaled, y=y, scoring='recall_weighted', cv=kFold, n_jobs=-1)
print('Mean score:', score.mean())

KeyboardInterrupt: 

The bagging algorithm has raised the bias and doesn't improve our model

## <font color='red'> Boosting

### Adaptive boosting with decision tree classifier

In [75]:
from sklearn.ensemble import AdaBoostClassifier

adaboost_clf = AdaBoostClassifier(base_estimator = dt_clf, learning_rate = 0.5)
adaboost_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(adaboost_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(adaboost_clf.score(X_test, y_test)))

Train score: 0.99
Test score: 0.65


We can see that adaptive boosting did in fact raise the average training accuracy 
for the Decision Tree but the test accuracy got reduced

### AdBoosting with Random Forest

In [76]:
from sklearn.ensemble import AdaBoostClassifier
adaboost_clf = AdaBoostClassifier(base_estimator = rf_clf, learning_rate = 0.5)
adaboost_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(adaboost_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(adaboost_clf.score(X_test, y_test)))

KeyboardInterrupt: 

We can see that adaptive boosting did in fact raise the average training accuracy for the Decision Tree but the test accuracy got reduced. It is still overfitting

### Adaptive Boosting with Logistic Regression

In [77]:
from sklearn.ensemble import AdaBoostClassifier

adaboost_clf = AdaBoostClassifier(base_estimator = lreg_clf, learning_rate = 0.5)
adaboost_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(adaboost_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(adaboost_clf.score(X_test, y_test)))

Train score: 0.57
Test score: 0.60


We can see that adaptive boosting improved our test accuracy with logistic model and has an improved model fit

### Adaptive Boosting with LinearSVC

In [78]:
from sklearn.ensemble import AdaBoostClassifier

adaboost_clf = AdaBoostClassifier(base_estimator = LSVC_clf, algorithm='SAMME')
adaboost_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(adaboost_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(adaboost_clf.score(X_test, y_test)))

Train score: 0.78
Test score: 0.72


We can see that adaptive boosting did in fact reduced the average test accuracy for the Decision Tree but the test accuracy got increased. Therefore, it has a poor fit with Linear SVC

### Adaptive Boosting with KNN
- KNeighborsClassifier does not support sample weights, so we will not be able to use Adaptive Boosting to lower the model bias.

### Adaptive Boosting with Kernel SVC

In [79]:
from sklearn.ensemble import AdaBoostClassifier

adaboost_clf = AdaBoostClassifier(base_estimator = KSVC_clf, algorithm='SAMME')
adaboost_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(adaboost_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(adaboost_clf.score(X_test, y_test)))

KeyboardInterrupt: 

We can see that adaptive boosting has very low accuracy for Kernel SVC. So it's not a good fit

### Gradient Boosting

In [ ]:
from sklearn.ensemble import GradientBoostingClassifier

gb_clf = GradientBoostingClassifier(learning_rate = 0.05)
gb_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(gb_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(gb_clf.score(X_test, y_test)))

Gradient Boosting is overfitting the model

In [ ]:
GB = GradientBoostingClassifier()
score = cross_val_score(estimator=GB, X=X_scaled, y=y, cv=kFold, n_jobs=-1)
gb_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(gb_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(gb_clf.score(X_test, y_test)))
print('Mean Accuracy:', score.mean())

Gradient Boosting is overfitting the model

In [ ]:
GB2 = GradientBoostingClassifier(min_samples_leaf=9, learning_rate=0.05, n_estimators=100)
score = cross_val_score(estimator=GB, X=X_scaled, y=y, cv=kFold, n_jobs=-1)
gb_clf.fit(X_train, y_train)
print('Train score: {0:0.2f}'.format(gb_clf.score(X_train, y_train)))
print('Test score: {0:0.2f}'.format(gb_clf.score(X_test, y_test)))
print('Mean Accuracy:', score.mean())

#### It is clear that, while the bagging and boosting techniques mentioned above are 
#### usually effective, most did not do much to improve the models.

- Due to reasons like imbalanced classes, high dimensionality and lack of observations, we couldn't get an optimal model.

## <font color='red'> Principal Component Analysis (PCA)

In [80]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100, svd_solver='auto')
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test)
X_comb_pca = np.concatenate((X_train_pca, X_test_pca), axis=0)

In [81]:
X_train_pca.shape

(384, 100)

## <font color='red'> Machine Learning Models with PCA

### KNN Classification with PCA

In [82]:
from sklearn.neighbors import KNeighborsClassifier

knn_clf = KNeighborsClassifier(n_jobs=-1)

param_grid={'n_neighbors':[1,2,3,4,5,7,10]}

grid_search = GridSearchCV(knn_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train_pca,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': [1, 2, 3, 4, 5, 7, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [83]:
grid_search.cv_results_

{'mean_fit_time': array([0.00339355, 0.00373526, 0.00351295, 0.00523491, 0.00412474,
        0.00372815, 0.00504613]),
 'mean_score_time': array([0.11380005, 0.11183434, 0.11402578, 0.1134593 , 0.11600695,
        0.11262069, 0.11519852]),
 'mean_test_score': array([0.5546875 , 0.58854167, 0.6015625 , 0.58854167, 0.58333333,
        0.56770833, 0.57291667]),
 'mean_train_score': array([1.        , 0.69723114, 0.67579153, 0.63796436, 0.62625759,
        0.60225821, 0.57490731]),
 'param_n_neighbors': masked_array(data=[1, 2, 3, 4, 5, 7, 10],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'n_neighbors': 1},
  {'n_neighbors': 2},
  {'n_neighbors': 3},
  {'n_neighbors': 4},
  {'n_neighbors': 5},
  {'n_neighbors': 7},
  {'n_neighbors': 10}],
 'rank_test_score': array([7, 2, 1, 2, 4, 6, 5]),
 'split0_test_score': array([0.58536585, 0.58536585, 0.59756098, 0.57317073, 0.57317073,
        0.54878049, 0.5365

In [84]:
grid_search.best_params_

{'n_neighbors': 3}

- Best Parameter : No. of neighbours=3

In [85]:
grid_search.best_score_

0.6015625

In [86]:
knn_clf=KNeighborsClassifier(n_neighbors=3)
knn_clf.fit(X_train_pca, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [87]:
y_pred_train = knn_clf.predict(X_train_pca)
y_pred_test = knn_clf.predict(X_test_pca)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.6770833333333334
Test Recall score: 0.6470588235294118


KNN shows an improved model accuracy and fit after applying PCA

### Logistic Regression with PCA

In [88]:
from sklearn.linear_model import LogisticRegression

lreg_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs')

param_grid = {'C': [0.00001,0.0001,0.001,0.01,0.1,1,10,100]}

grid_search = GridSearchCV(lreg_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train_pca,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [89]:
grid_search.cv_results_

{'mean_fit_time': array([0.11031833, 0.10719943, 0.07631512, 0.03818297, 0.02939463,
        0.03852129, 0.07358351, 0.0974535 ]),
 'mean_score_time': array([0.00069385, 0.00091662, 0.00110354, 0.00130067, 0.00128832,
        0.00148611, 0.00131269, 0.00101109]),
 'mean_test_score': array([0.53385417, 0.53385417, 0.53385417, 0.53385417, 0.609375  ,
        0.70572917, 0.6875    , 0.64583333]),
 'mean_train_score': array([0.53393884, 0.53393884, 0.53393884, 0.53393884, 0.64196531,
        0.82880072, 0.95507739, 0.9973978 ]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001},
  {'C': 0.01},
  {'C': 0.1},
  {'C': 1},
  {'C': 10},
  {'C': 100}],
 'rank_test_score': array([5, 5, 5, 5, 4, 1, 2, 3]),
 'split0_test_score': array([0.5       , 0.5       , 0.5       , 0.5       , 0.597

In [90]:
grid_search.best_params_

{'C': 1}

- Best parameter : C=1

In [91]:
grid_search.best_score_

0.7057291666666666

In [92]:
lreg_clf = LogisticRegression(multi_class='multinomial', solver='lbfgs',C=1)
lreg_clf.fit(X_train_pca, y_train)

LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='multinomial',
          n_jobs=1, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, verbose=0, warm_start=False)

In [93]:
y_pred_train = lreg_clf.predict(X_train_pca)
y_pred_test = lreg_clf.predict(X_test_pca)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.8203125
Test Recall score: 0.6764705882352942


Logistics Regression shows an improved model accuracy and fit after applying PCA. 

### Linear SVM with PCA

In [94]:
from sklearn.svm import LinearSVC

LSVC_clf = LinearSVC(multi_class='crammer_singer')

param_grid = {'C': [0.00001,0.0001,0.001,0.01,0.1,1,10,100]}

grid_search = GridSearchCV(LSVC_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train_pca,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [95]:
grid_search.cv_results_

{'mean_fit_time': array([0.0082274 , 0.01229348, 0.02872224, 0.03746872, 0.04009824,
        0.11683884, 0.38283691, 0.46555071]),
 'mean_score_time': array([0.00225077, 0.00186863, 0.00100436, 0.00149975, 0.00176778,
        0.0011878 , 0.00130968, 0.00140285]),
 'mean_test_score': array([0.6171875 , 0.61458333, 0.61458333, 0.6171875 , 0.69791667,
        0.70052083, 0.63020833, 0.59114583]),
 'mean_train_score': array([0.64643744, 0.64514494, 0.64448269, 0.65356614, 0.77480555,
        0.91137569, 0.98760332, 1.        ]),
 'param_C': masked_array(data=[1e-05, 0.0001, 0.001, 0.01, 0.1, 1, 10, 100],
              mask=[False, False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'C': 1e-05},
  {'C': 0.0001},
  {'C': 0.001},
  {'C': 0.01},
  {'C': 0.1},
  {'C': 1},
  {'C': 10},
  {'C': 100}],
 'rank_test_score': array([4, 6, 6, 4, 2, 1, 3, 8]),
 'split0_test_score': array([0.63414634, 0.62195122, 0.62195122, 0.62195122, 0.682

In [96]:
grid_search.best_params_

{'C': 1}

- Best parameter : C=1

In [97]:
grid_search.best_score_

0.7005208333333334

In [98]:
LSVC_clf = LinearSVC(multi_class='crammer_singer', C=1)
LSVC_clf.fit(X_train_pca, y_train)

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='crammer_singer', penalty='l2', random_state=None,
     tol=0.0001, verbose=0)

In [99]:
y_pred_train = LSVC_clf.predict(X_train_pca)
y_pred_test = LSVC_clf.predict(X_test_pca)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.90625
Test Recall score: 0.6911764705882353


Linear SVM still overfits the model after applying PCA. 

### Kernalised SVM with PCA

In [100]:
from sklearn.multiclass import OneVsOneClassifier
from sklearn import svm

KSVC_clf = svm.SVC(kernel='rbf')

param_grid = {'C': [0.0001,0.001,0.01,0.1,1,10],
          'gamma': [0.0001,0.001,0.01,0.1,1,10]}

grid_search = GridSearchCV(KSVC_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train_pca,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'C': [0.0001, 0.001, 0.01, 0.1, 1, 10], 'gamma': [0.0001, 0.001, 0.01, 0.1, 1, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [101]:
grid_search.cv_results_

{'mean_fit_time': array([0.02329803, 0.0260983 , 0.0290565 , 0.02382956, 0.02944183,
        0.02440333, 0.0264667 , 0.02654343, 0.02495589, 0.0258193 ,
        0.02739878, 0.03085971, 0.02205687, 0.0229527 , 0.0263104 ,
        0.0353972 , 0.07219791, 0.08513908, 0.02503438, 0.02531457,
        0.02713728, 0.04462214, 0.08973494, 0.12169266, 0.02132483,
        0.03607206, 0.03710036, 0.04484215, 0.11353149, 0.12918143,
        0.03735271, 0.03912077, 0.03170319, 0.04793549, 0.11100459,
        0.12312293]),
 'mean_score_time': array([0.00608106, 0.00779424, 0.00708632, 0.00582104, 0.00602326,
        0.00573044, 0.00624123, 0.00632792, 0.00571537, 0.0059813 ,
        0.00621748, 0.00603247, 0.00539956, 0.00482039, 0.00549788,
        0.00641723, 0.00794048, 0.00711451, 0.00592742, 0.00571718,
        0.00548635, 0.00619373, 0.00919356, 0.00772805, 0.00502434,
        0.00610871, 0.00691032, 0.00663772, 0.00780749, 0.00787063,
        0.00731897, 0.00671263, 0.00600281, 0.00732794, 0.

In [102]:
grid_search.best_params_

{'C': 10, 'gamma': 0.1}

- Best parameters : C=0.0001, gamma=1

In [103]:
grid_search.best_score_

0.7083333333333334

In [104]:
from sklearn import svm
KSVC_clf = svm.SVC(kernel='rbf',C=10,gamma=0.1)
KSVC_clf.fit(X_train_pca, y_train)

SVC(C=10, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma=0.1, kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [105]:
y_pred_train = KSVC_clf.predict(X_train_pca)
y_pred_test = KSVC_clf.predict(X_test_pca)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.9661458333333334
Test Recall score: 0.6764705882352942


Kernalized SVM still overfits the model after applying PCA.

### Decision Trees with PCA

In [106]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier()
param_grid = {'max_depth': [2, 3, 4, 5,6, 10, 20]}

grid_search = GridSearchCV(dt_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train_pca,y_train)

GridSearchCV(cv=StratifiedKFold(n_splits=5, random_state=None, shuffle=False),
       error_score='raise',
       estimator=DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [2, 3, 4, 5, 6, 10, 20]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='recall_weighted', verbose=0)

In [107]:
grid_search.cv_results_

{'mean_fit_time': array([0.01455064, 0.02011843, 0.02934551, 0.03300323, 0.03734202,
        0.06519251, 0.0579257 ]),
 'mean_score_time': array([0.00124216, 0.00110126, 0.00103359, 0.00140519, 0.00120311,
        0.00236754, 0.00131073]),
 'mean_test_score': array([0.53385417, 0.52864583, 0.53385417, 0.5390625 , 0.50260417,
        0.43489583, 0.43229167]),
 'mean_train_score': array([0.58802314, 0.63098482, 0.70465248, 0.77037599, 0.82632373,
        0.94215768, 1.        ]),
 'param_max_depth': masked_array(data=[2, 3, 4, 5, 6, 10, 20],
              mask=[False, False, False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'max_depth': 2},
  {'max_depth': 3},
  {'max_depth': 4},
  {'max_depth': 5},
  {'max_depth': 6},
  {'max_depth': 10},
  {'max_depth': 20}],
 'rank_test_score': array([2, 4, 2, 1, 5, 6, 7]),
 'split0_test_score': array([0.45121951, 0.43902439, 0.45121951, 0.45121951, 0.40243902,
        0.26829268, 0.34146341]),
 'spli

In [108]:
grid_search.best_params_

{'max_depth': 5}

- Best parameter : max_depth = 4

In [109]:
grid_search.best_score_

0.5390625

In [110]:
dt_clf = DecisionTreeClassifier(max_depth=4)
dt_clf.fit(X_train_pca, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [111]:
y_pred_train = dt_clf.predict(X_train_pca)
y_pred_test = dt_clf.predict(X_test_pca)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Train Recall score: 0.703125
Test Recall score: 0.5441176470588235


- We can see that decision tree performs really bad with PCA. This may be due to information lose due to dimentionality reduction.
- Also model is also overfitting as there is difference between train and test score.

### Random Forest with PCA

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf_clf = RandomForestClassifier(n_estimators=300, criterion='gini',n_jobs= -1,random_state=10)

param_grid = {'max_features': [25,50,75,100],
          'max_depth': [6,8,10,12,14],
           'max_leaf_nodes':[20,22,30,50]}

grid_search = GridSearchCV(rf_clf, param_grid, scoring = 'recall_weighted',cv=kFold, return_train_score=True)
grid_search.fit(X_train_pca,y_train)

In [ ]:
grid_search.cv_results_

In [ ]:
grid_search.best_params_

- Best parameters : max_features=50, max_depth=12, max_leaf_nodes=50

In [ ]:
grid_search.best_score_

In [ ]:
rf_clf = RandomForestClassifier(n_estimators=300, criterion='gini',max_features=50,max_depth=12,max_leaf_nodes=50)
rf_clf.fit(X_train_pca, y_train)

In [ ]:
y_pred_train = rf_clf.predict(X_train_pca)
y_pred_test = rf_clf.predict(X_test_pca)

print('Train Recall score: {}'
      .format(recall_score(y_train, y_pred_train, average='weighted')))
print('Test Recall score: {}'
      .format(recall_score(y_test, y_pred_test, average='weighted')))

Random forest is also overfitting the model after applying PCA.

### Conclusion

The Results have improved after using PCA. We get the best resulting using Linear SVC after applying PCA

## Effort Table

In [112]:
import pandas as pd
effort = { 'Name':["Mehul Patel","Shaurya Trehan"],'Effort':["50%","50%"]}
effort_table = pd.DataFrame(effort)
effort_table = effort_table[['Name', 'Effort']]
effort_table

,Name,Effort
0,Mehul Patel,50%
1,Shaurya Trehan,50%
